In [5]:
import numpy as np
import scipy as sc 
import pandas as pd 
from scipy.integrate import quad
from scipy.spatial.distance import squareform
from numpy import random
import networkx as nx
import pickle

from tv.data_load import load_data
import os
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit

import matplotlib.pyplot as plt
%matplotlib inline

path = '/home/bgutman/datasets/HCP/'
connec , thinkness, log_jac, unique_labels, labels, mean_area_eq, idx_subj_connec, idx_subj_think, idx_subj_logjac = load_data(path)


Connectomes:  (789, 2, 68, 68)
Thinkness:  (789, 2, 163842)
Log Jacob:  (789, 2, 163842)
Unique Labels:  (789, 2, 163842)
Mean Labels:  (2, 163842)
Mean Area eq:  (2, 163842)


In [ ]:
np.where(labels[0]!=0)[0].shape, np.where(labels[1]!=0)[0].shape, 

In [84]:
name = '_LogJac_2e-4_eq.raw'
log_jac_rnd_subj = []
for subj_id in rnd_idx:
    with open(path_log_jac + subj_id + '/' + 'LH' + name, 'rb') as f:
        l_log = np.fromfile(f, count=-1 ,dtype='float32')
    with open(path_log_jac + subj_id + '/' + 'RH' + name, 'rb') as f:
        r_log = np.fromfile(f, count=-1 ,dtype='float32')
    log_jac_rnd_subj += [np.concatenate([l_log, r_log])]
log_jac_rnd_subj[0][:10]

array([ 0.10931737,  0.00248825, -0.2810666 ,  0.26209646,  0.39557737,
       -0.12322377,  0.29820508,  0.10148587, -0.04911281,  0.03229546], dtype=float32)

In [86]:
name = '_LogJac_2e-4.raw'
log_jac_rnd_subj = []
for subj_id in rnd_idx:
    with open(path_log_jac + subj_id + '/' + 'LH' + name, 'rb') as f:
        l_log = np.fromfile(f, count=-1 ,dtype='float32')
    with open(path_log_jac + subj_id + '/' + 'RH' + name, 'rb') as f:
        r_log = np.fromfile(f, count=-1 ,dtype='float32')
    log_jac_rnd_subj += [np.concatenate([l_log, r_log])]
log_jac_rnd_subj[0][:10]

array([ 0.15088511,  0.0441572 , -0.24098271,  0.30375329,  0.43716973,
       -0.08146474,  0.33986503,  0.14321454, -0.0074365 ,  0.07392117], dtype=float32)

In [85]:
name = '_LogJac_no_smooth.raw'
log_jac_rnd_subj = []
for subj_id in rnd_idx:
    with open(path_log_jac + subj_id + '/' + 'LH' + name, 'rb') as f:
        l_log = np.fromfile(f, count=-1 ,dtype='float32')
    with open(path_log_jac + subj_id + '/' + 'RH' + name, 'rb') as f:
        r_log = np.fromfile(f, count=-1 ,dtype='float32')
    log_jac_rnd_subj += [np.concatenate([l_log, r_log])]
log_jac_rnd_subj[0][:10]

array([ 0.16003117,  0.12447903, -0.22958869,  0.25944901,  0.40776467,
       -0.08842522,  0.37073737,  0.22918081, -0.26744515,  0.08832674], dtype=float32)

In [87]:
name = '_LogJac_no_smooth_eq.raw'
log_jac_rnd_subj = []
for subj_id in rnd_idx:
    with open(path_log_jac + subj_id + '/' + 'LH' + name, 'rb') as f:
        l_log = np.fromfile(f, count=-1 ,dtype='float32')
    with open(path_log_jac + subj_id + '/' + 'RH' + name, 'rb') as f:
        r_log = np.fromfile(f, count=-1 ,dtype='float32')
    log_jac_rnd_subj += [np.concatenate([l_log, r_log])]
log_jac_rnd_subj[0][:10]

array([ 0.11844908,  0.08272374, -0.27125239,  0.21779856,  0.36620718,
       -0.13027602,  0.32908732,  0.1874184 , -0.30914211,  0.04677144], dtype=float32)

In [6]:
rnd_idx = np.array(['212823', '633847', '201818', '770352', '111312', '201414',
       '614439', '453441', '175338', '177746'])
rnd_idx

path_thick = '/home/bgutman/datasets/HCP/FreeSurfer/thickness/'
path_log_jac = '/home/bgutman/datasets/HCP/FreeSurfer/LogJacobian/'

thick_rnd_subj = []
for subj_id in rnd_idx:
    with open(path_thick + subj_id + '/' + 'LH_thick_smooth_2e-4.raw', 'rb') as f:
        l_th = np.fromfile(f, count=-1 ,dtype='float32')
        l_th = np.where(l_th < 0, 0, l_th)
    
    with open(path_thick + subj_id + '/' + 'RH_thick_smooth_2e-4.raw', 'rb') as f:
        r_th = np.fromfile(f, count=-1 ,dtype='float32')
        r_th = np.where(r_th < 0, 0, r_th)
    thick_rnd_subj += [np.concatenate([l_th, r_th])]
    

thick_rnd_subj = np.array(thick_rnd_subj)
thick_rnd_subj.shape

log_jac_rnd_subj = []
for subj_id in rnd_idx:
    with open(path_log_jac + subj_id + '/' + 'LH_LogJac_2e-4_eq.raw', 'rb') as f:
        l_log = np.fromfile(f, count=-1 ,dtype='float32')
    with open(path_log_jac + subj_id + '/' + 'RH_LogJac_2e-4_eq.raw', 'rb') as f:
        r_log = np.fromfile(f, count=-1 ,dtype='float32')
    log_jac_rnd_subj += [np.concatenate([l_log, r_log])]
print(log_jac_rnd_subj[0][:10])
log_jac_rnd_subj = -np.array(log_jac_rnd_subj)
log_jac_rnd_subj.shape
mean_area_eq = []
with open('/home/bgutman/datasets/HCP/' + 'LH_mean_area.raw', 'rb') as f:
    mean_area_eq += list(np.fromfile(f, count=-1 ,dtype='float32'))
with open('/home/bgutman/datasets/HCP/' + 'RH_mean_area.raw', 'rb') as f:
    mean_area_eq += list(np.fromfile(f, count=-1 ,dtype='float32'))

mean_area_eq = np.array(mean_area_eq)

labels = labels.reshape(-1)
areas = (mean_area_eq * np.exp(log_jac_rnd_subj))[:,idx_0].sum(axis = -1)
volumes = (mean_area_eq * np.exp(log_jac_rnd_subj)*thick_rnd_subj)[:,idx_0].sum(axis= -1)

for i in range(len(rnd_idx)):
    print('idx {}  area {} vol {}'.format(rnd_idx[i], areas[i], volumes[i]))

[ 0.10931737  0.00248825 -0.2810666   0.26209646  0.39557737 -0.12322377
  0.29820508  0.10148587 -0.04911281  0.03229546]


NameError: name 'idx_0' is not defined

In [7]:
def compute_area_volume(s_id, path):
    
    # load Log Jacobian
    local_dir = 'FreeSurfer/LogJacobian/{}/'.format(s_id)
    filename = ['LH_LogJac_2e-4.raw','RH_LogJac_2e-4.raw']
    
    
    
    LH_Jacobian = np.fromfile('{}{}{}'.format(path, 
                                              local_dir,
                                              filename[0]),
                              dtype='float32')
    
    RH_Jacobian = np.fromfile('{}{}{}'.format(path, 
                                              local_dir,
                                              filename[1]),
                              dtype='float32')
    # load area template
    LH_area_template = np.fromfile('{}{}'.format(path, 'LH_mean_area.raw'),
                                   dtype='float32')
    RH_area_template = np.fromfile('{}{}'.format(path, 'RH_mean_area.raw'),
                                   dtype='float32')
    
    LH_labels = np.fromfile('{}{}'.format(path, 'LH_labels_MajVote.raw'),
                                   dtype='float32')
    
    RH_labels = np.fromfile('{}{}'.format(path, 'RH_labels_MajVote.raw'),
                                   dtype='float32')
    
    LH_labels = np.where(LH_labels > 0, 1, 0)
    RH_labels = np.where(RH_labels > 0, 1, 0)
    
    
    # load thickness 
    local_dir = 'FreeSurfer/thickness/{}/'.format(s_id)
    filename = ['LH_thick_smooth_2e-4.raw','RH_thick_smooth_2e-4.raw']
    
       
    LH_thick = np.fromfile('{}{}{}'.format(path,
                                           local_dir,
                                           filename[0]),
                          dtype='float32')
    RH_thick = np.fromfile('{}{}{}'.format(path,
                                           local_dir,
                                           filename[1]),
                          dtype='float32')
    
    area = (np.exp(LH_Jacobian) * LH_area_template * LH_labels).sum() + \
    (np.exp(RH_Jacobian) * RH_area_template * RH_labels).sum()
        
    volume = (np.exp(LH_Jacobian) * LH_area_template * LH_thick * LH_labels).sum() + \
    (np.exp(RH_Jacobian) * RH_area_template * RH_thick * RH_labels).sum()
    
    return area, volume

In [8]:
PATH = '/home/bgutman/datasets/HCP/'

In [9]:
for subject in rnd_idx:
    area, volume = compute_area_volume(subject, PATH)
    print('s_id: {}, area: {}, vol: {}'.format(subject,area, volume))

s_id: 212823, area: 141791.7789734751, vol: 367066.78901723924
s_id: 633847, area: 132819.8247544989, vol: 345176.10007971834
s_id: 201818, area: 118851.45641564205, vol: 324977.3467972155
s_id: 770352, area: 128706.21440098435, vol: 340348.9438486902
s_id: 111312, area: 122803.71250853315, vol: 308567.76620729326
s_id: 201414, area: 127562.36112456024, vol: 333515.2210892493
s_id: 614439, area: 122088.34744047746, vol: 327528.94541855535
s_id: 453441, area: 138462.26440973952, vol: 349508.5503493956
s_id: 175338, area: 152127.04454194382, vol: 369978.4706571568
s_id: 177746, area: 124062.51114761084, vol: 341055.28749491984


In [35]:
idx_nodes = np.unique(labels)[1:]
idx_nodes

array([  1.,   2.,   3.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,  12.,
        13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,
        24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,
        35.,  36.,  37.,  38.,  40.,  41.,  42.,  43.,  44.,  45.,  46.,
        47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,
        58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,
        69.,  70.], dtype=float32)

In [38]:
idx_0 = np.where(labels!=0)[0]

In [44]:
i = 0
for one in (mean_area_eq * np.exp(log_jac_rnd_subj))[:,idx_0].sum(axis = -1):
    print('subj {} area {}'.format(rnd_idx[i], one))
    i+=1

subj 212823 area 94981.75
subj 633847 area 96699.21875
subj 201818 area 95795.0390625
subj 770352 area 95812.0625
subj 111312 area 97349.453125
subj 201414 area 96793.8046875
subj 614439 area 97260.8125
subj 453441 area 94802.2109375
subj 175338 area 92645.15625
subj 177746 area 97645.5


In [45]:
i = 0
for one in (mean_area_eq * np.exp(log_jac_rnd_subj)*thick_rnd_subj)[:,idx_0].sum(axis = -1):
    print('subj {} vol {}'.format(rnd_idx[i], one))
    i +=1

subj 212823 vol 245882.625
subj 633847 vol 251324.9375
subj 201818 vol 261945.828125
subj 770352 vol 253356.796875
subj 111312 vol 244648.515625
subj 201414 vol 253058.4375
subj 614439 vol 260917.546875
subj 453441 vol 239302.453125
subj 175338 vol 225320.734375
subj 177746 vol 268432.9375


In [46]:
(mean_area_eq * np.exp(log_jac_rnd_subj)*thick_rnd_subj)[:,idx_0]

(10, 302310)

In [3]:
idx_nodes = list(range(1,4)) + list(range(5,39)) + list(range(40,71))
idx_nodes = np.array(idx_nodes)
print(idx_nodes)
print(np.unique(labels))

[ 1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26
 27 28 29 30 31 32 33 34 35 36 37 38 40 41 42 43 44 45 46 47 48 49 50 51 52
 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70]
[  0.   1.   2.   3.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.  30.
  31.  32.  33.  34.  35.  36.  37.  38.  40.  41.  42.  43.  44.  45.  46.
  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.  57.  58.  59.  60.  61.
  62.  63.  64.  65.  66.  67.  68.  69.  70.]


In [4]:
thinkness = thinkness.reshape(789,-1)
log_jac = log_jac.reshape(789,-1)
labels = labels.reshape(-1).astype(int)
mean_area_eq = mean_area_eq.reshape(-1)

In [5]:
roi_thick = np.array([np.mean(thinkness[:,np.where(labels==node)[0]], axis = -1) for node in idx_nodes]).T


In [6]:
roi_area = np.array([np.sum(mean_area_eq[np.where(labels==node)[0]]*np.exp(log_jac[:,np.where(labels==node)[0]]),
                             axis = -1) 
                             for node in idx_nodes]).T

In [7]:
roi_vol = np.array([np.sum(mean_area_eq[np.where(labels==node)[0]]*np.exp(log_jac[:,np.where(labels==node)[0]]) * thinkness[:,np.where(labels == node)[0]],
                             axis = -1) 
                             for node in idx_nodes]).T

print(roi_area.shape, roi_thick.shape,roi_vol.shape)


(789, 68) (789, 68) (789, 68)


In [ ]:
with open('meta_HCP_for_Ayagoz.csv', 'rb') as f:
    meta = pd.read_csv(f)

In [ ]:
meta['Gender'] = meta['Gender'].astype('category')
meta['Age'] = meta['Age'].astype('category')
meta['Age'] = meta['Age'].cat.codes
meta["Gender"] = meta['Gender'].cat.codes

In [ ]:
idx_women = np.where(np.array(meta.Gender) == 0)[0]
idx_men = np.where(np.array(meta.Gender) == 1)[0]

In [ ]:
plt.hist(roi_area[idx_men])
plt.hist(roi_area[idx_women], alpha  = 0.7);

In [ ]:
plt.hist(roi_vol[idx_men])
plt.hist(roi_vol[idx_women], alpha  = 0.7);

In [8]:
rnd_idx

array([406, 619, 369, 685,  39, 367, 608, 529, 275, 286])

In [12]:
path_b = '/home/bgutman/datasets/HCP/Connectomes/'
name = '_NxNmatrix_FULL_NORM.txt'
idx_nodes = np.array(list(range(3)) + list(range(4,38)) + list(range(39, 70)))
rnd_10_subj_thick = []
for idx in idx_subj_connec[rnd_idx]:
    with open(path_b+ idx + '/con/' + idx + name, 'rb') as f:
        t1 = np.loadtxt(f, dtype='float32')
        np.fill_diagonal(t1,0)
        t1 = t1[np.ix_(idx_nodes,idx_nodes)]
    rnd_10_subj_thick += [t1]

In [23]:
path_t = '/home/bgutman/datasets/HCP/FreeSurfer/thickness/'
with open(path_t + idx1 + '/' + 'LH_thick_smooth_2e-4.raw', 'rb') as f:
    th1 = np.fromfile(f, count=-1 ,dtype='float32')
    th1 = np.where(th1 < 0, 0, th1)

In [24]:
np.allclose(thinkness[453, 0, :], th1)

True

In [25]:
with open(path_t + idx2 + '/' + 'LH_thick_smooth_2e-4.raw', 'rb') as f:
    th2 = np.fromfile(f, count=-1 ,dtype='float32')
    th2 = np.where(th2 < 0, 0, th2)

In [26]:
np.allclose(thinkness[788, 0, :], th2)

True

In [35]:
path_l = '/home/bgutman/datasets/HCP/FreeSurfer/LogJacobian/'
with open(path_l + idx1 + '/' + 'LH_LogJac_2e-4_eq.raw', 'rb') as f:
    l1 = np.fromfile(f, count=-1 ,dtype='float32')

In [36]:
np.allclose(log_jac[453, 1, :], l1)

True